In [ ]:
!pip install openai
!pip install streamlit
!pip install torch accelerate
!pip install transformers==4.43.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-no

In [ ]:
from google.colab import output
from huggingface_hub import notebook_login

output.enable_custom_widget_manager()
notebook_login()

In [ ]:
%cd /content/drive/MyDrive/KU/2024-여름

/content/drive/MyDrive/KU/2024-여름


In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import openai
from openai import OpenAI
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

data_path = '/content/drive/MyDrive/KU/2024-여름'
model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model.eval()

# 텍스트 파일에서 데이터 추출
def extract_tarot_descriptions_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        df_tarot = pd.read_csv(file_path, encoding = 'cp949')

    # 카드 명칭과 설명을 매칭하는 로직
    cards = ["나그네", "전술사", "난이", "왕비", "임금", "스승", "연인", "거북선", "용기",
             "현인", "운명", "정의", "매달린 사람", "저승", "절제", "도깨비", "붕괴", "별",
             "달", "해", "업보", "세계"]

    descriptions = {}
    for card in cards:
        descriptions[card] = df_tarot[df_tarot['카드']==card]['설명'].values[0]
    return descriptions

# 검색 인덱스 생성
def create_faiss_index(descriptions):
    tarot_df = pd.DataFrame(list(descriptions.items()), columns=["label", "description"])

    return tarot_df#, vectorizer, index

def retrieve_card_descriptions(query, tarot_df, k=3):
    return tarot_df[tarot_df['label'] == query]['description'].values[0]

def get_interpretation_nolbu(card_descriptions):
    input_data = "\n".join([f"Card {i+1}: {desc}" for i, desc in enumerate(card_descriptions)])
    system_message = f"The user has selected the following tarot cards:\n{input_data}\n" \
                     "Based on these cards and the user's query, provide a detailed interpretation." \
                     "Answer in Korean"\
                     "말투는 싸가지 없는 말투로 해줘. 일단 욕설과 함께 반말을 하고, '네놈'이라는 호칭을 사용해줘."\
                     "'엣헴!'이라는 말투도 중간중간 적절히 추가해줘."\
                     "카드의 의미를 다 합쳐서 최종적으로 정리해주는 것도 #총정리 로 해줘."
    return system_message

def get_interpretation_heungbu(card_descriptions):
    input_data = "\n".join([f"Card {i+1}: {desc}" for i, desc in enumerate(card_descriptions)])
    system_message = f"The user has selected the following tarot cards:\n{input_data}\n" \
                     "Based on these cards and the user's query, provide a detailed interpretation." \
                     "Answer in Korean"\
                     "말투는 아주 착한 말투로 해줘. 무조건 존댓말, '손님'이라는 호칭을 사용해줘."\
                     "'헤헤헤'라는 웃음과 조선시대와 같은 말투도 중간중간 적절히 추가해줘."\
                     "각 카드가 무슨 카드인지 설명해주고 세 카드의 의미를 다 합쳐서 최종적으로 정리해주는 것도 #총정리 로 해줘."
    return system_message

def get_response(query, system_message, client=None, flag = True):

    if flag:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "user", "content": query},
                {"role": "system", "content": system_message}
            ]
        )
        return response.choices[0].message.content

    else:
        messages=[
            {"role": "system", "content": f"{system_message}"},
            {"role": "user", "content": f"{query}"}
        ]
        inputs = tokenizer.apply_chat_template(messages,
                                               add_generation_prompt=True,
                                               return_tensors="pt").to(model.device)
        terminators = [
            tokenizer.eos_token_id,
            tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]

        outputs = model.generate(
            inputs,
            max_new_tokens=1024,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            repetition_penalty = 1.1
        )
        description = tokenizer.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True)
        print(description)
        return description

def main():
    # Streamlit 앱
    st.header("흥부 놀부 타로 챗봇에 오신걸 환영합니다.")
    st.write("타로 카드 3개를 골라주세요.")

    # OpenAI API Key
    openai_api_key = st.text_input("Enter OpenAI API Key", type="password")
    if not openai_api_key:
        st.info("Please add your OpenAI API key to continue.", icon="🔑")
        #st.info("LLaMa 3.1이 사용됩니다.")
        #flag = False

    else:
        client = OpenAI(api_key=openai_api_key)
        try:
            client.models.list()
            flag = True

        except openai.AuthenticationError as e:
            flag = False

        if flag:
            st.info("OpenAI API Key added successfully.", icon="✅")

        else:
            st.info("OpenAI API Key가 유효하지 않습니다. LLaMa3-Bllossom이 사용됩니다.")
            client = None

        # 텍스트 파일 경로 입력 받기
        csv_path = data_path + '/한국형타로카드설명.csv'

        if csv_path:
            try:
                # 텍스트 파일에서 데이터 추출
                tarot_descriptions = extract_tarot_descriptions_from_txt(csv_path)
                tarot_df = create_faiss_index(tarot_descriptions)

                # User selects 3 tarot cards
                selected_cards = st.multiselect("Select 3 tarot cards", list(tarot_df['label']))

                if len(selected_cards) == 3:
                    card_descriptions = [retrieve_card_descriptions(card, tarot_df)[0] for card in selected_cards]
                    # User inputs their query
                    query = st.text_input("Ask your question related to the tarot reading")

                    if query:
                        character = st.selectbox("흥부와 놀부 중 누구한테 타로점을 보시겠습니까?", ["흥부", "놀부"])
                        if character == "흥부":
                            system_message = get_interpretation_heungbu(card_descriptions)
                            interpretation = get_response(query, system_message, client, flag)
                        else:
                            system_message = get_interpretation_nolbu(card_descriptions)
                            interpretation = get_response(query, system_message, client, flag)
                        print(interpretation)
                        st.text_area("Tarot Reading Interpretation", value=interpretation, height=200)
            except Exception as e:
                st.error(f"An error occurred: {e}")

if __name__ == '__main__':
    main()

Writing app.py


In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!npm install localtunnel



added 22 packages, and audited 23 packages in 3s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))


In [ ]:
!streamlit run app.py &>/dev/null&


In [ ]:
!npx localtunnel --port 8501

your url is: https://giant-friends-tell.loca.lt
^C
